# ✅ Pré-processamento
Este notebook carrega e normaliza automaticamente duas bases:
- `VGG19_256_MAX.csv`
- `VGG19_256_MAX_PCA.csv`

Todas as etapas dos comitês (Bagging, Boosting, Random Forest) são executadas nas **duas bases**.

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

# Bases a normalizar
bases = [
    'VGG19_256_MAX.csv',
    'VGG19_256_MAX_PCA.csv'
]

# Dicionário para armazenar (X_normalizado, y)
bases_normalizadas = {}

for base in bases:
    print(f"\n🔄 Normalizando: {base}")

    # Carregar base
    df = pd.read_csv(base)

    # Separar atributos e classe
    X = df.drop(columns=['classe'])
    y = df['classe'].astype(str)  # <- IMPORTANTE para evitar erros com tipos mistos

    # Normalizar os atributos
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # Salvar em novo DataFrame
    df_scaled = pd.DataFrame(X_scaled, columns=X.columns)
    df_final = pd.concat([df_scaled, y], axis=1)

    # Salvar CSV normalizado
    nome_saida = base.replace('.csv', '_NORMALIZADA.csv')
    df_final.to_csv(nome_saida, index=False)
    print(f"✅ Base salva como: {nome_saida}")

    # Armazenar para uso posterior
    bases_normalizadas[base.replace('.csv', '')] = (df_scaled, y)

print("\n🎉 Todas as bases foram normalizadas, salvas e estão prontas para uso.")



🔄 Normalizando: VGG19_256_MAX.csv
✅ Base salva como: VGG19_256_MAX_NORMALIZADA.csv

🔄 Normalizando: VGG19_256_MAX_PCA.csv
✅ Base salva como: VGG19_256_MAX_PCA_NORMALIZADA.csv

🎉 Todas as bases foram normalizadas, salvas e estão prontas para uso.


# 🔷 BAGGING Experimentos

In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import cross_val_score, cross_val_predict, train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
import numpy as np
import pandas as pd

estimadores = [
    ('Decision Tree', DecisionTreeClassifier()),
    ('k-NN', KNeighborsClassifier()),
    ('Naive Bayes', GaussianNB()),
    ('MLP', MLPClassifier(max_iter=500))
]

n_estimators_list = [10, 20, 30]
resultados = []

for nome_base, (X, y) in bases_normalizadas.items():
    for nome_clf, clf in estimadores:
        for n in n_estimators_list:
            model = BaggingClassifier(estimator=clf, n_estimators=n, random_state=1)

            # 10-fold cross-validation
            acc_cv = cross_val_score(model, X, y, cv=10, scoring='accuracy')
            y_pred_cv = cross_val_predict(model, X, y, cv=10)
            cm_cv = confusion_matrix(y, y_pred_cv)

            # Split 70/30
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, shuffle=True)
            model.fit(X_train, y_train)
            y_pred_split = model.predict(X_test)
            acc_split = accuracy_score(y_test, y_pred_split)
            cm_split = confusion_matrix(y_test, y_pred_split)

            # Salvar resultados
            resultados.append({
                'Base': nome_base,
                'Classificador': nome_clf,
                'n_estimators': n,
                'Acuracia_10fold': round(np.mean(acc_cv)*100, 2),
                'Desvio_10fold': round(np.std(acc_cv)*100, 2),
                'Matriz_Confusao_10fold': cm_cv.tolist(),
                'Acuracia_Split7030': round(acc_split*100, 2),
                'Matriz_Confusao_Split7030': cm_split.tolist()
            })

# Exportar para CSV
df_resultados = pd.DataFrame(resultados)
df_resultados.to_csv('Resultados_Bagging.csv', index=False)

print("✅ Resultados salvos em 'Resultados_Bagging.csv'")


✅ Resultados salvos em 'Resultados_Bagging.csv'


In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score, cross_val_predict, train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

# Simule ou carregue seu dicionário com as bases já normalizadas
# bases_normalizadas = {'nome_base': (X, y)}

# Parâmetros
max_features_list = [0.3, 0.5, 0.8]
n_estimators_list = [10, 20, 30]
estimadores = [
    ('Decision Tree', DecisionTreeClassifier()),
    ('k-NN', KNeighborsClassifier()),
    ('Naive Bayes', GaussianNB()),
]

# Dicionário para guardar resultados por max_features
resultados_por_max_features = {
    0.3: [],
    0.5: [],
    0.8: []
}

# Loop nos experimentos
for nome_base, (X, y) in bases_normalizadas.items():
    y = y.astype(str)

    for nome_clf, clf in estimadores:
        for max_feat in max_features_list:
            for n in n_estimators_list:
                model = BaggingClassifier(
                    estimator=clf,
                    n_estimators=n,
                    max_features=max_feat,
                    random_state=1
                )

                # 10-fold CV
                acc_cv = cross_val_score(model, X, y, cv=10, scoring='accuracy')
                y_pred_cv = cross_val_predict(model, X, y, cv=10)
                cm_cv = confusion_matrix(y, y_pred_cv)

                # Hold-out 70/30
                X_train, X_test, y_train, y_test = train_test_split(
                    X, y, test_size=0.3, random_state=42, shuffle=True
                )
                model.fit(X_train, y_train)
                y_pred_split = model.predict(X_test)
                acc_split = accuracy_score(y_test, y_pred_split)
                cm_split = confusion_matrix(y_test, y_pred_split)

                # Armazena os resultados na lista correta
                resultados_por_max_features[max_feat].append({
                    'Base': nome_base,
                    'Classificador': nome_clf,
                    'n_estimators': n,
                    'max_features': max_feat,
                    'Acuracia_10fold': round(acc_cv.mean() * 100, 2),
                    'Desvio_10fold': round(acc_cv.std() * 100, 2),
                    'Matriz_Confusao_10fold': cm_cv.tolist(),
                    'Acuracia_Split7030': round(acc_split * 100, 2),
                    'Matriz_Confusao_Split7030': cm_split.tolist()
                })

# Exportar para 3 arquivos CSV separados
for max_feat, resultados in resultados_por_max_features.items():
    df = pd.DataFrame(resultados)
    nome_arquivo = f'Resultados_Bagging_FeatureSelection_maxfeat{str(max_feat).replace(".", "")}.csv'
    df.to_csv(nome_arquivo, index=False)
    print(f"✅ CSV salvo: {nome_arquivo}")


✅ CSV salvo: Resultados_Bagging_FeatureSelection_maxfeat03.csv
✅ CSV salvo: Resultados_Bagging_FeatureSelection_maxfeat05.csv
✅ CSV salvo: Resultados_Bagging_FeatureSelection_maxfeat08.csv


# 🔷 BOOSTING Experimentos

In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict

# Lista para armazenar resultados
boosting_resultados = []

# Classificadores a testar com AdaBoost
estimadores = [
    ('AD', DecisionTreeClassifier()),
    ('NB', GaussianNB())
]

# Loop nas bases e parâmetros
for nome_base, (X, y) in bases_normalizadas.items():
    y = y.astype(str)  # Evita erro com tipos mistos

    for nome_clf, clf in estimadores:
        for n in [10, 20, 30]:
            model = AdaBoostClassifier(estimator=clf, n_estimators=n, random_state=1)

            # 🔁 10-fold cross-validation
            acc_cv = cross_val_score(model, X, y, cv=10, scoring='accuracy')
            y_pred_cv = cross_val_predict(model, X, y, cv=10)
            cm_cv = confusion_matrix(y, y_pred_cv)

            # 🎯 Hold-out (70/30)
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1, shuffle=True)
            model.fit(X_train, y_train)
            y_pred_split = model.predict(X_test)
            acc_split = accuracy_score(y_test, y_pred_split)
            cm_split = confusion_matrix(y_test, y_pred_split)

            # Salvar resultado
            boosting_resultados.append({
                'Base': nome_base,
                'Classificador': nome_clf,
                'n_estimators': n,
                'Acuracia_10fold': round(np.mean(acc_cv) * 100, 2),
                'Desvio_10fold': round(np.std(acc_cv) * 100, 2),
                'Matriz_Confusao_10fold': cm_cv.tolist(),
                'Acuracia_Split7030': round(acc_split * 100, 2),
                'Matriz_Confusao_Split7030': cm_split.tolist()
            })

# Exportar para CSV
df_resultados = pd.DataFrame(boosting_resultados)
df_resultados.to_csv('Resultados_Boosting_Holdout_e_KFold.csv', index=False)

print("✅ Resultados salvos em 'Resultados_Boosting_Holdout_e_KFold.csv'")


✅ Resultados salvos em 'Resultados_Boosting_Holdout_e_KFold.csv'


In [ ]:
## Carregando o Bagging e os algoritmos base (DT), (MLP), (k-NN) e (NB)
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics
from sklearn.metrics import confusion_matrix

In [ ]:
## Preparando Hold-out de 70/30
from sklearn.model_selection import train_test_split

# 70% training and 30% test
X_train_70, X_test_30, y_train_70, y_test_30 = train_test_split(X, y, test_size=0.3, random_state=1)

## Instanciando Boosting com DecisionTree
adaclassifier = AdaBoostClassifier(estimator=DecisionTreeClassifier(),
                                   n_estimators=10,learning_rate=1.0)

adaclassifier.fit(X_train_70,y_train_70)
y_pred = adaclassifier.predict(X_test_30)

## Model Accuracy
acuracia = metrics.accuracy_score(y_test_30, y_pred)
print('Accuracy: %.3f' % acuracia)

# Matriz de confusão p/ 30%
confusion_matrix(y_test_30, y_pred)

Accuracy: 0.693


array([[116,  30],
       [ 41,  44]])

In [ ]:
## Preparando 10-fold CV
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict

# 10-fold CV
kf = KFold(n_splits=10, random_state=1, shuffle=True)

## Instanciando Boosting com DecisionTree
adaclassifier = AdaBoostClassifier(estimator=DecisionTreeClassifier(),
                                   n_estimators=10,learning_rate=1.0)

# Model Accuracy
scores = cross_val_score(adaclassifier, X, y, scoring='accuracy', cv=kf)
print('Accuracy: %.3f (%.3f)' % (scores.mean(), scores.std()))

# Matriz de confusão p/ k fold
y_pred = cross_val_predict(adaclassifier, X, y, cv=kf)
confusion_matrix(y, y_pred)

Accuracy: 0.698 (0.067)


array([[378, 122],
       [111, 157]])

# 🔷 RANDOMFOREST Experimentos

In [ ]:
## Carregando o Random Forest
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import confusion_matrix

In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Substitua isso pela sua estrutura de dados já normalizada
# Exemplo:
# bases_normalizadas = {
#     'VGG19_256_MAX': (X1, y1),
#     'VGG19_256_MAX_PCA': (X2, y2),
# }

# Parâmetros
criterios = ['gini', 'entropy', 'log_loss']
n_estimators_list = [10, 20, 30, 100]

# Lista para armazenar os resultados
resultados_rf = []

# Loop pelas bases normalizadas
for nome_base, (X, y) in bases_normalizadas.items():
    y = y.astype(str)  # prevenir erros de tipo

    for criterio in criterios:
        linha_resultado = {
            'Base': nome_base,
            'Criterio': criterio
        }

        accs = []

        for n_estimators in n_estimators_list:
            model = RandomForestClassifier(
                n_estimators=n_estimators,
                criterion=criterio,
                random_state=42
            )
            scores = cross_val_score(model, X, y, cv=10, scoring='accuracy')
            media = round(scores.mean() * 100, 2)
            desvio = round(scores.std() * 100, 2)
            linha_resultado[f'Acc_{n_estimators}'] = media
            accs.append(media)

        linha_resultado['Media_Acc'] = round(np.mean(accs), 2)
        resultados_rf.append(linha_resultado)

# Gerar DataFrame final
df_rf = pd.DataFrame(resultados_rf)

# Salvar como CSV
df_rf.to_csv('Resultados_RandomForest_Colab.csv', index=False)
print("✅ Resultados salvos em 'Resultados_RandomForest_Colab.csv'")


✅ Resultados salvos em 'Resultados_RandomForest_Colab.csv'


# 🧠 Stacking Classifier (Heterogêneo)
Executa comitês de stacking combinando diferentes classificadores base.
Variações: 5, 10, 15, 20 classificadores com `VotingClassifier` ou `StackingClassifier`.

A base final de decisão pode ser uma árvore, MLP ou regressão logística.

In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score

# Substitua com suas bases normalizadas
# Exemplo:
# bases_normalizadas = {
#     'VGG19_256_MAX': (X1, y1),
#     'VGG19_256_MAX_PCA': (X2, y2),
# }

# Classificadores base a serem usados no Stacking
modelos_base = [
    ('knn', KNeighborsClassifier()),
    ('dt', DecisionTreeClassifier()),
    ('nb', GaussianNB())
]

# Número de classificadores no stacking
quantidades = [5, 10, 15, 20]

# Lista para armazenar os resultados
resultados_stacking = []

for nome_base, (X, y) in bases_normalizadas.items():
    y = y.astype(str)  # Evitar erro com tipos mistos

    for qnt in quantidades:
        # Repete os modelos até atingir a quantidade desejada
        estimadores = [
            (f'{nome}_{i}', clf)
            for i, (nome, clf) in enumerate(modelos_base * ((qnt // len(modelos_base)) + 1))
        ][:qnt]

        # Define o stacking
        stacking = StackingClassifier(
            estimators=estimadores,
            final_estimator=LogisticRegression(),
            cv=10
        )

        # Avaliação com 10-fold
        scores = cross_val_score(stacking, X, y, cv=10, scoring='accuracy')

        resultados_stacking.append({
            'Base': nome_base,
            'Classificadores_Base': qnt,
            'Acuracia_Media': round(scores.mean() * 100, 2),
            'Desvio_Padrao': round(scores.std() * 100, 2)
        })

# Gerar DataFrame final e salvar
df_stacking = pd.DataFrame(resultados_stacking)
df_stacking.to_csv('Resultados_Stacking_Colab.csv', index=False)
print("✅ Resultados salvos em 'Resultados_Stacking_Colab.csv'")


✅ Resultados salvos em 'Resultados_Stacking_Colab.csv'
